<a href="https://colab.research.google.com/github/SilvanaJ90/usergioarboleda-bootcamp_IA/blob/main/Semana_2_Clase_viernes_Mejoramiento_de_red_neuronal_profunda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Redes profundas.

# 1. Introducción. 🏳

La semana pasada nos dedicamos a introducir redes neuronales y fracasamos en la
clasificación de imágenes del set datos CIFAR10. La verdad es que esto fue principalmente porque la red neuronal estaba extraordinariamente sencilla, no era lo suficientemente profunda, no tenían ni las suficientes capas, ni las suficientes neuronas.


Lo que vamos a estar haciendo es corregir los errores que cometimos con CIFAR10 con una Red Neuronal Profunda, aunque nos vamos a encontrar con 3 problemas principales que tenemos que aprender a resolver. El primer problema consiste en la desaparición de los gradientes a través de las diferentes capas. También se presenta la lentitud a la hora del entrenamiento debido al aumento de información. Así mismo, está el constante problema del sobreajuste.


*   Como objetivos del capítulo se establece arreglar la red neuronal para la clasificación
imágenes de CIFAR10 y aprender las herramientas necesarias para entender publicaciones
sobre las investigaciones de redes neuronales.




# 2. Mejorando red neuronal profunda para CIFAR10.

Vamos a comenzar armando el modelo que más o menos hicimos la vez pasada, le vamos a estar colgando diferentes inicializaciones, diferentes funciones de activación, batch normalización y muchas otras cosas. Comencemos importando las librerías que necesitamos.

# 2.1 Importando librerias relevantes.

In [1]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow import keras
import os

# 3. Extrayendo los datos del CIFAR-10.

In [ ]:
(X_entrena_completo, y_entrena_completo), (X_test,y_test) = keras.datasets.cifar10.load_data()
# Imagenes entrenamiento, etiquetas de imagenes de entrenamiento - imagenes validación, etiquetas de imagenes de validación

170498071/170498071 [==============================] - 3s 0us/step


# 4. Creando nuestra semilla (Seed) para conservar concordancia en la aleatoriedad.

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

# 5. Modelo secuencial.

# 5.1 Creando el modelo.

Se aplican la sespecificaciones indicadas al inicio. 20 capas intermedias con 100 neuronas cada capa intermedia, una capa de entrada y una de salida con 10 nodos o neuronas.

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32,32,3]))
for _ in range(20):
  model.add(keras.layers.Dense(100,activation="relu"))
model.add(keras.layers.Dense(10,activation="softmax"))

# 5.2 Resumen del modelo recien definido.

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 3072)              0         
                                                                 
 dense (Dense)               (None, 100)               307300    
                                                                 
 dense_1 (Dense)             (None, 100)               10100     
                                                                 
 dense_2 (Dense)             (None, 100)               10100     
                                                                 
 dense_3 (Dense)             (None, 100)               10100     
                                                                 
 dense_4 (Dense)             (None, 100)               10100     
                                                                 
 dense_5 (Dense)             (None, 100)               1

# 5.3 Compilando el modelo.

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

# 6.0 Early stopping

Para evitar el sobre entrenamiento de una red neuronal profunda, donde se llegue a un punto entre las transiciones de las epocas de la misma en que ya la precisión no mejora y el error no disminuye, se estableceran unos 'puntos de guardado' o 'puntos de validación'. Para ello debemos modificar la cantidad los elementos de nuestro set de datos de entrenamiento y de valdación. Tambien, se deben estructurar los callbacks que nos permitiran la aplicación del early stopping.

In [ ]:
X_train = X_entrena_completo[5000:]
y_train = y_entrena_completo[5000:]
# Datos desde el indice 5000 hasta el ultimo

X_valid = X_entrena_completo[:5000]
y_valid = y_entrena_completo[:5000]
# Datos desde el inicio hasta el 4999.

In [ ]:
#Callback a 20 puntos, guardadndo en modelocifar10.h5
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("modelocifar10.h5", save_best_only=True)
callbacks=[early_stopping_cb,model_checkpoint_cb]

# 7. Entrenamiento del modelo usando early stopping. 😓 🚦

In [ ]:
history = model.fit(X_train,y_train, epochs = 30, validation_data=(X_valid, y_valid))

Epoch 1/30
1407/1407 [==============================] - 14s 7ms/step - loss: 2.0762 - accuracy: 0.2072 - val_loss: 2.0363 - val_accuracy: 0.2334
Epoch 2/30
1407/1407 [==============================] - 9s 7ms/step - loss: 1.8995 - accuracy: 0.2950 - val_loss: 1.8481 - val_accuracy: 0.3118
Epoch 3/30
1407/1407 [==============================] - 10s 7ms/step - loss: 1.8192 - accuracy: 0.3324 - val_loss: 2.4412 - val_accuracy: 0.2212
Epoch 4/30
1407/1407 [==============================] - 10s 7ms/step - loss: 1.7568 - accuracy: 0.3617 - val_loss: 1.9178 - val_accuracy: 0.3090
Epoch 5/30
1407/1407 [==============================] - 10s 7ms/step - loss: 1.7148 - accuracy: 0.3802 - val_loss: 1.7589 - val_accuracy: 0.3468
Epoch 6/30
1407/1407 [==============================] - 10s 7ms/step - loss: 1.6795 - accuracy: 0.3952 - val_loss: 1.8189 - val_accuracy: 0.3538
Epoch 7/30
1407/1407 [==============================] - 10s 7ms/step - loss: 1.6444 - accuracy: 0.4078 - val_loss: 1.6805 - val_acc

# 7.1 Evaluación de nuestro modelo recien entrenado. 🧪

In [ ]:
model.evaluate(X_test,y_test)

313/313 [==============================] - 1s 3ms/step - loss: 1.7759 - accuracy: 0.4091


[1.7759257555007935, 0.4090999960899353]



---



# 8. Limpiando la sesion anterior de entrenamiento.

In [ ]:
keras.backend.clear_session()

# 9. Definiendo nuevamente nuestra semilla.

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

*Se notara la diferencia a partir de este punto, donde se aplicaran los conceptos vistos en el material guia en la plataforma*

# 10. Definiendo el modelo con unos cambios.

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32,32,3]))
for _ in range(20):
  model.add(keras.layers.Dense(100,activation="elu", kernel_initializer="he_normal"))
model.add(keras.layers.Dense(10,activation="softmax"))

# 10.1 Resumen del nuevo modelo aplicando la inicialización he_normal en las capas intermedias.

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 3072)              0         
                                                                 
 dense (Dense)               (None, 100)               307300    
                                                                 
 dense_1 (Dense)             (None, 100)               10100     
                                                                 
 dense_2 (Dense)             (None, 100)               10100     
                                                                 
 dense_3 (Dense)             (None, 100)               10100     
                                                                 
 dense_4 (Dense)             (None, 100)               10100     
                                                                 
 dense_5 (Dense)             (None, 100)               1

# 10.1 Compilar el modelo (no se usa el early stopping).


In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

# 10.2 Iniciando el entrenamiento del nuevo modelo.

In [ ]:
history.model.fit(X_train,y_train, epochs=30,validation_data=(X_valid,y_valid))

Epoch 1/30
1407/1407 [==============================] - 10s 7ms/step - loss: 1.3082 - accuracy: 0.5314 - val_loss: 1.5039 - val_accuracy: 0.4788
Epoch 2/30
1407/1407 [==============================] - 10s 7ms/step - loss: 1.2973 - accuracy: 0.5379 - val_loss: 1.6867 - val_accuracy: 0.4096
Epoch 3/30
1407/1407 [==============================] - 9s 7ms/step - loss: 1.2948 - accuracy: 0.5361 - val_loss: 1.5511 - val_accuracy: 0.4638
Epoch 4/30
1407/1407 [==============================] - 10s 7ms/step - loss: 1.2799 - accuracy: 0.5439 - val_loss: 1.5278 - val_accuracy: 0.4740
Epoch 5/30
1407/1407 [==============================] - 10s 7ms/step - loss: 1.2737 - accuracy: 0.5448 - val_loss: 1.6240 - val_accuracy: 0.4496
Epoch 6/30
1407/1407 [==============================] - 10s 7ms/step - loss: 1.2678 - accuracy: 0.5478 - val_loss: 1.5034 - val_accuracy: 0.4704
Epoch 7/30
1407/1407 [==============================] - 10s 7ms/step - loss: 1.2587 - accuracy: 0.5520 - val_loss: 1.5979 - val_acc

# 10.3 Evaluando el entrenamiento de nuestro nuevo modelo.

In [ ]:
model.evaluate(X_test,y_test)

313/313 [==============================] - 1s 3ms/step - loss: 231.6206 - accuracy: 0.1001


[231.62062072753906, 0.10010000318288803]



---



# 11. Ajustando el modelo sobreajustado.

# 11.1 Limpiando la sesion anterior.

In [ ]:
keras.backend.clear_session()

# 11.2 Definiendo las semillas

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

# 11.3 Armando el nuevo modelo (por tercera vez) 😆.

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32,32,3]))
for _ in range(20):
 model.add(keras.layers.Dense(100, activation = "elu", kernel_initializer="he_normal"))
model.add(keras.layers.Dense(10, activation= "softmax"))

# 11.4 Cambiando el optimizador usado.

In [ ]:
optimizer = keras.optimizers.Nadam(learning_rate=5e-5,beta_1=0.9, beta_2=0.999)


# 11.5 Compilando el modelo

In [ ]:
# NOTESE LA DIFERENCIA ACA.
model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer, metrics=["accuracy"])


# 11.6 apicando early stopping

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("modelocifar10.h5", save_best_only=True)
callbacks=[early_stopping_cb,model_checkpoint_cb]

# 11.7 Empezamos el entrenamiento con optimizador Nadam.

In [ ]:
model.fit(X_train,y_train,epochs=100,validation_data=(X_valid,y_valid),callbacks=callbacks)

Epoch 1/100
1407/1407 [==============================] - 26s 11ms/step - loss: 3.4808 - accuracy: 0.1797 - val_loss: 2.1313 - val_accuracy: 0.2444
Epoch 2/100
   6/1407 [..............................] - ETA: 15s - loss: 2.2370 - accuracy: 0.2083

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1407/1407 [==============================] - 15s 11ms/step - loss: 2.0468 - accuracy: 0.2581 - val_loss: 2.0048 - val_accuracy: 0.2618
Epoch 3/100
1407/1407 [==============================] - 15s 10ms/step - loss: 1.9275 - accuracy: 0.3042 - val_loss: 1.8874 - val_accuracy: 0.3088
Epoch 4/100
1407/1407 [==============================] - 15s 11ms/step - loss: 1.8441 - accuracy: 0.3311 - val_loss: 1.8451 - val_accuracy: 0.3334
Epoch 5/100
1407/1407 [==============================] - 15s 11ms/step - loss: 1.7936 - accuracy: 0.3526 - val_loss: 1.7687 - val_accuracy: 0.3602
Epoch 6/100
1407/1407 [==============================] - 15s 11ms/step - loss: 1.7414 - accuracy: 0.3776 - val_loss: 1.7079 - val_accuracy: 0.3754
Epoch 7/100
1407/1407 [==============================] - 15s 10ms/step - loss: 1.6959 - accuracy: 0.3912 - val_loss: 1.7107 - val_accuracy: 0.3772
Epoch 8/100
1407/1407 [==============================] - 15s 11ms/step - loss: 1.6641 - accuracy: 0.4034 - val_loss: 1.6331 - val_

# 11.8 Verificar el modelo luego del entrenamiento con el optimizador Nadam.

In [ ]:
model = keras.models.load_model("modelocifar10.h5")
model.evaluate(X_valid,y_valid)

157/157 [==============================] - 1s 3ms/step - loss: 1.4988 - accuracy: 0.4794


[1.4988383054733276, 0.47940000891685486]

# 12. Creando el nuevo modelo aplciando normalización batch y capa de actuivación relu en cada capa intermedia.

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32,32,3]))
model.add(keras.layers.BatchNormalization())
for _ in range(20):
 model.add(keras.layers.Dense(100, activation = "elu", kernel_initializer="he_normal"))
 model.add(keras.layers.BatchNormalization())
 model.add(keras.layers.Activation("elu"))
model.add(keras.layers.Dense(10, activation= "softmax"))


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 3072)              0         
                                                                 
 batch_normalization (Batch  (None, 3072)              12288     
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 100)               307300    
                                                                 
 batch_normalization_1 (Bat  (None, 100)               400       
 chNormalization)                                                
                                                                 
 activation (Activation)     (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               1

# 13. Optimización y compilación del nuevo modelo.

In [ ]:
optimizer = keras.optimizers.Nadam(learning_rate=5e-4, beta_1=.9, beta_2=.999)
model.compile(loss="sparse_categorical_crossentropy",  optimizer=optimizer, metrics=["accuracy"])

# 14. Aplicando el early stopping

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("modelocifar10.h5",save_best_only=True)
callbacks=[early_stopping_cb,model_checkpoint_cb]


# 15. Empezando el entrenamiento del modelo aplicando batch normalization, early stopping y capa de activacion elu en las capas intermedias de la red neuronal profunda.

In [ ]:

model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=callbacks)


Epoch 1/100
1407/1407 [==============================] - ETA: 0s - loss: 2.0419 - accuracy: 0.2559

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1407/1407 [==============================] - 64s 30ms/step - loss: 2.0419 - accuracy: 0.2559 - val_loss: 1.7814 - val_accuracy: 0.3510
Epoch 2/100
1407/1407 [==============================] - 41s 29ms/step - loss: 1.7967 - accuracy: 0.3483 - val_loss: 1.6931 - val_accuracy: 0.3842
Epoch 3/100
1407/1407 [==============================] - 40s 29ms/step - loss: 1.7013 - accuracy: 0.3921 - val_loss: 1.6120 - val_accuracy: 0.4134
Epoch 4/100
1407/1407 [==============================] - 39s 28ms/step - loss: 1.6372 - accuracy: 0.4141 - val_loss: 1.5529 - val_accuracy: 0.4358
Epoch 5/100
1407/1407 [==============================] - 40s 28ms/step - loss: 1.5916 - accuracy: 0.4350 - val_loss: 1.5010 - val_accuracy: 0.4636
Epoch 6/100
1407/1407 [==============================] - 38s 27ms/step - loss: 1.5432 - accuracy: 0.4531 - val_loss: 1.5051 - val_accuracy: 0.4514
Epoch 7/100
1407/1407 [==============================] - 39s 28ms/step - loss: 1.5015 - accuracy: 0.4656 - val_loss: 1.4702 - val_

# 16. Evaluando el modelo.

In [ ]:
model = keras.models.load_model("modelocifar10.h5")
model.evaluate(X_valid,y_valid)

157/157 [==============================] - 1s 4ms/step - loss: 1.3269 - accuracy: 0.5298


[1.326931118965149, 0.5297999978065491]